In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

block_size = 8 #
batch_size = 4 #
max_iterations = 10000 #
learning_rate = 3e-3 #
evaluate_iterations = 250 #

Using device: cpu


In [13]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars =  sorted(set(text))

vocabulary_size = len(chars)

print(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [14]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([91, 48, 66, 77, 69, 62, 26,  1, 32, 72, 75, 72, 77, 65, 82,  1, 58, 71,
        61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,  1, 66, 71,  1, 43, 83,
         0,  0, 29, 78, 77, 65, 72, 75, 26,  1, 40, 14,  1, 34, 75, 58, 71, 68,
         1, 30, 58, 78, 70,  0,  0, 37, 69, 69, 78, 76, 77, 75, 58, 77, 72, 75,
        26,  1, 38, 72, 65, 71,  1, 46, 14,  1, 42, 62, 66, 69, 69,  0,  0, 46,
        62, 69, 62, 58, 76, 62,  1, 61, 58, 77])


In [15]:
# split the data into train and validation sets
n=int(0.8*len(data)) #0.8
train_data = data[:n]
val_data = data[n:]

# function to generate a batch of data
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)
print('targets:')
print(y.shape)
print(y)

'''
https://www.youtube.com/watch?v=BTVUPhpXUd4&list=PLOZ8y37SEZTbf_57K1I4g7Jjrr6raTxbk&index=5
https://www.youtube.com/watch?v=Ip6NkFQ5DKw
https://www.youtube.com/watch?v=2FYzX2eW_aw

'''

inputs:
torch.Size([4, 8])
tensor([[78, 71, 61,  1, 58, 71, 61,  1],
        [58, 76,  1, 82, 72, 78,  1, 58],
        [62, 77,  1, 65, 62,  0, 70, 78],
        [63, 72, 75,  1, 77, 65, 62,  1]])
targets:
torch.Size([4, 8])
tensor([[71, 61,  1, 58, 71, 61,  1, 63],
        [76,  1, 82, 72, 78,  1, 58, 75],
        [77,  1, 65, 62,  0, 70, 78, 76],
        [72, 75,  1, 77, 65, 62,  1, 73]])


'\nhttps://www.youtube.com/watch?v=BTVUPhpXUd4&list=PLOZ8y37SEZTbf_57K1I4g7Jjrr6raTxbk&index=5\nhttps://www.youtube.com/watch?v=Ip6NkFQ5DKw\nhttps://www.youtube.com/watch?v=2FYzX2eW_aw\n\n'

In [16]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding__table = nn.Embedding(vocab_size, 32)

    def forward(self, index, targets=None):

        logits = self.token_embedding_table(index)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        # logits = self.token_embedding_table(index)  # (B,T,C)
        # B,T,C = logits.shape
        # logits = logits.view(B*T, C)
        # targets = targets.view(B*T)
        # loss = F.cross_entropy(logits, targets.view(B*T)) if targets is not None else None
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocabulary_size)
m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)


In [17]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(evaluate_iterations)
        for k in range(evaluate_iterations):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [18]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iterations):
    if iter % evaluate_iterations == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



AttributeError: 'GPTLanguageModel' object has no attribute 'token_embedding_table'